Global step
-----------

https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/training/training_util.py

其实就是一个计数的变量，用来记录训练的步数，而不少的函数都会依赖这个变量。

这行代码就可以创建名字叫global_step的变量，并且不参与训练。它的值只能显示修改，而不会被optimizer更新。

`global_step_tensor = tf.Variable(10, trainable=False, name='global_step')`

以下几个相关的函数：

* tf.train.get_global_step()：得到global_step_tensor对象
* tf.train.get_or_create_global_step()
* tf.train.create_global_step()
* tf.train.global_step()：其实就是sess.run(global_step_tensor)

这个变量一开始是没有的，create_global_step()会创建这个命名变量，并加入到GLOBAL_STEP和GLOBAL_VARIABLES这两个collection。

In [1]:
import tensorflow as tf

print tf.train.get_global_step()
print repr(tf.GraphKeys.GLOBAL_STEP)
print tf.get_collection(tf.GraphKeys.GLOBAL_STEP)

None
'global_step'
[]


In [2]:
print tf.train.get_or_create_global_step()

<tf.Variable 'global_step:0' shape=() dtype=int64_ref>


In [3]:
print tf.train.get_global_step()
print tf.get_collection(tf.GraphKeys.GLOBAL_STEP)

<tf.Variable 'global_step:0' shape=() dtype=int64_ref>
[<tf.Variable 'global_step:0' shape=() dtype=int64_ref>]


# tf.assign_add(ref, value)

https://www.tensorflow.org/api_docs/python/tf/assign_add

就是`+=`操作，第一个参数的tensor的引用，第二个是需要加的值。

In [5]:
step = tf.train.get_global_step()
incr = tf.assign_add(step, 1)

sess = tf.Session()
sess.run(tf.global_variables_initializer())
sess.run(incr)
sess.run(incr)

sess.run(step)

2

# 使用

tensorflow中很多函数都会有global_step这个可选的参数。

例1：tf.train.Optimizer.minimize()这个函数会返回一个train_step，实际上是计算梯度并更新模型，如果参数给定了global_step，这个函数会调用assign_add对步数加1。

例2：tf.train.Saver.save()也接收一个可选的global_step参数，如果给定了步数，会生成步数对应的checkpoint文件。

https://www.tensorflow.org/api_docs/python/tf/train/Saver

    saver.save(sess, 'my-model', global_step=0) ==> filename: 'my-model-0'
    ...
    saver.save(sess, 'my-model', global_step=1000) ==> filename: 'my-model-1000'
    
例3：tf.train.MonitoredSession

在使用这个session的时候，如果设置了checkpoint_dir，会自动添加一个tf.train.StepCounterHook。而这个hook会周期性的在一段时间或者一定的步骤更新summary。但如果创建session的时候，没有global_step这个变量，会抛出以下异常。

    RuntimeError: Global step should be created to use StepCounterHook.
    
tf.train.CheckpointSaverHook会定期（时间或者步数）更新checkpoint。而这个global_step变量最终会传递到tf.Saver.save()方法上，在不同的步数生成不同的文件。